<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 0.5
<br>
<b>Deadline:</b> March 2, 2020 (Monday). 23:00
</div>

# Exercise 1.1. Logistic regression

The goal of this exercise is to get familiar with the basics of PyTorch and train a simple logistic regression model.

If you are not familiar with PyTorch, there is a number of good tutorials [here](https://pytorch.org/tutorials/index.html). We recommend the following ones:
* [What is PyTorch?](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py)
* [Autograd: Automatic Differentiation](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)
* [Learning PyTorch with Examples](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
* [Neural Networks](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py)

In [1]:
skip_training = True  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F

import tools
import data

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select device which you are going to use for training
#device = torch.device("cuda:0")
device = torch.device("cpu")

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

# Data

We are going to use *winequality* dataset which contains red and white vinho verde wine samples rated by experts from 0 to 10 (obtained from [here](https://archive.ics.uci.edu/ml/datasets/wine+quality)).

In [7]:
trainset = data.WineQuality(data_dir, train=True, normalize=False)
train_inputs, train_targets = trainset.tensors
print(train_inputs.shape, train_targets.shape)

testset = data.WineQuality(data_dir, train=False, normalize=False)
test_inputs, test_targets = testset.tensors

torch.Size([5197, 11]) torch.Size([5197])


We will transform the task into a binary classification problem and try to predict if the quality of wine is greater or lower than 7.

In [8]:
# Convert to a binary classification problem
train_targets = (train_targets >= 7).float().view(-1, 1)  
test_targets = (test_targets >= 7).float().view(-1, 1)  

The optimization problem is often easier when the model inputs are normalized to zero mean and unit variance.

In [13]:
# Normalize inputs to zero mean and unit variance
mean = train_inputs.mean(dim=0)
std = train_inputs.std(dim=0)
scaler = lambda x: (x - mean.to(x.device)) / std.to(x.device)

train_inputs = scaler(train_inputs)
test_inputs = scaler(test_inputs)

# Logistic regression classifier

Logistic regression is a linear model which is used to solve a binary classification task. According to the model, the probability that example $x$ belongs to class 1 is computed as
$$
  p(y=1 \mid x) = \sigma (w^T x + b)
$$
where vector $w$ and scalar $b$ are the parameters of the model and $\sigma(\cdot)$ is the sigmoid function.

In the cell below, your task is to specify the logistic regression model as a PyTorch module.

In [26]:
class LogisticRegression(nn.Module):
    def __init__(self, n_inputs=11):
        # YOUR CODE HERE
        super(LogisticRegression, self).__init__()
        self.n_inputs = n_inputs
        self.w = nn.Linear(n_inputs, 1, bias=True)
#         raise NotImplementedError()

    def forward(self, x):
        """
        Args:
          x of shape (n_samples, n_inputs): Model inputs.
        
        Returns:
          y of shape (n_samples, 1): Model outputs.
        """
        # YOUR CODE HERE
        out = torch.sigmoid(self.w(x))
        return out
#         raise NotImplementedError()

In [27]:
# Let us create the network and make sure it can process a random input of the right shape
def test_logreg_shapes():
    n_inputs = 11
    n_samples = 10
    model = LogisticRegression()
    print(model)
    y = model(torch.randn(n_samples, n_inputs))
    assert y.shape == torch.Size([n_samples, 1]), f"Bad y.shape: {y.shape}"
    print('Success')

test_logreg_shapes()

LogisticRegression(
  (w): Linear(in_features=11, out_features=1, bias=True)
)
Success


# Train the model

Next we will train the logistic regression model. The model is trained by minimizing the following loss:
$$
  \text{loss} = \sum_i - [ t_i \log(y_i) + (1−t_i) \log(1−y_i)]
$$
where $t_i$ is the target class label and $y_i$ is the output of the logistic regression classifier for training sample $x_i$.
This loss function is implemented in function [`binary_cross_entropy`](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.binary_cross_entropy) of PyTorch.

### Training loop

Your task is to implement the training loop.
You may find it useful to look at [this tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py).
Your should have the following steps:
* Set all gradient values to zeros.
* Calculate the output of the model for all training examples.
* Calculate the binary cross entropy loss (see [`binary_cross_entropy`](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.binary_cross_entropy)).
* Backpropagate the gradients: compute the gradients of the loss wrt to all the parameters of the model.
* Update the parameters of the model using the chosen optimizer.

Recommended hyperparameters:
* [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate 0.001.
* You can process the data in the full-batch model (computing the gradients using all training data).
* Number of iterations (parameter updates): 8000.

Hints:
- We recommend you to print the classification accuracy during training. You can compute the accuracy using function `compute_accuracy`.
- The accuracy on the training set should be above 0.8. The accuracy on the test set should be above 0.79.

In [29]:
# Compute the accuracy of the model on the given dataset
def compute_accuracy(model, inputs, targets):
    with torch.no_grad():
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = (model.forward(inputs) > 0.5).float()
        accuracy = (outputs == targets).sum().float() / targets.numel()
        return accuracy

In [30]:
# Create the model
model = LogisticRegression()
model.to(device)

LogisticRegression(
  (w): Linear(in_features=11, out_features=1, bias=True)
)

In [41]:
model.parameters()

<generator object Module.parameters at 0x7f6c8b0a2ed0>

In [42]:
model.w.reset_parameters()

In [44]:
# Implement the training loop in this cell
if not skip_training:
    # YOUR CODE HERE
    
    criterion = nn.functional.binary_cross_entropy
    optim = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(8000):
        optim.zero_grad()
        out = model.forward(train_inputs)
        loss = criterion(out, train_targets)
        if epoch%100==0:
            print("epoch: {}, loss: {}, acc: {}".format(epoch+1, loss, compute_accuracy(model,train_inputs,train_targets)))
        loss.backward()
        optim.step()
#     raise NotImplementedError()

epoch: 1, loss: 0.4283190071582794, acc: 0.8183567523956299

epoch: 101, loss: 0.41835731267929077, acc: 0.8172022104263306

epoch: 201, loss: 0.4109708368778229, acc: 0.8175870776176453

epoch: 301, loss: 0.4052406847476959, acc: 0.8214354515075684

epoch: 401, loss: 0.40080833435058594, acc: 0.8241292834281921

epoch: 501, loss: 0.39739757776260376, acc: 0.8258610963821411

epoch: 601, loss: 0.3947867453098297, acc: 0.8248989582061768

epoch: 701, loss: 0.3928021490573883, acc: 0.8252838253974915

epoch: 801, loss: 0.3913048207759857, acc: 0.8248989582061768

epoch: 901, loss: 0.39018577337265015, acc: 0.8248989582061768

epoch: 1001, loss: 0.389356404542923, acc: 0.8250914216041565

epoch: 1101, loss: 0.3887494206428528, acc: 0.8248989582061768

epoch: 1201, loss: 0.3883114755153656, acc: 0.8250914216041565

epoch: 1301, loss: 0.3879990577697754, acc: 0.8250914216041565

epoch: 1401, loss: 0.38778045773506165, acc: 0.8247065544128418

epoch: 1501, loss: 0.38763004541397095, acc: 0.8

In [46]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    tools.save_model(model, '1_logreg.pth')
else:
    model = LogisticRegression()
    tools.load_model(model, '1_logreg.pth', device)

Do you want to save the model (type yes to confirm)? yes
Model saved to 1_logreg.pth.


In [47]:
accuracy = compute_accuracy(model, test_inputs, test_targets)
print('Accuracy on test set:', accuracy.item())
assert accuracy >= 0.79, 'Logistic regression classifier has poor accuracy.'
print('Success')

Accuracy on test set: 0.8061538338661194
Success


In [ ]:
# This cell tests LogisticRegression